In [153]:
import pandas as pd
import json 

from src.utils import get_parameters_combinations,get_param_value_union,get_promt_text_params,get_df_with_clothing_promt


with open('data/female-style-type.json') as f:
    female_style_type = json.load(f)
with open('data/male-style-type.json') as f:
    male_style_type = json.load(f)
with open('data/female-types-with-parameters.json') as f:
    female_types_with_parameters = json.load(f)
with open('data/female-types.json') as f:
    female_types = json.load(f)
with open('data/male-types-with-parameters.json') as f:
    male_types_with_parameters = json.load(f)
with open('data/male-types.json') as f:
    male_types = json.load(f)

#layered_top
female_layered_top_df=pd.read_csv('data/female_layered_top.csv',index_col=0)
male_layered_top_df=pd.read_csv('data/male_layered_top.csv',index_col=0)
#
female_style_type_df=pd.DataFrame(female_style_type).explode('types').reset_index(drop=True)
male_style_type_df=pd.DataFrame(male_style_type).explode('types').reset_index(drop=True)
#
female_df_clothing_promt=get_df_with_clothing_promt(gender_types=female_types,gender_types_with_parameters=female_types_with_parameters)
male_df_clothing_promt=get_df_with_clothing_promt(gender_types=male_types,gender_types_with_parameters=male_types_with_parameters)
#
df_female_style_promt=female_style_type_df.merge(female_df_clothing_promt,left_on='types',right_on='type')
df_male_style_promt=male_style_type_df.merge(male_df_clothing_promt,left_on='types',right_on='type')
#



In [154]:
#
df_female_style_promt=df_female_style_promt.merge(female_layered_top_df,on='type',how='left')
df_male_style_promt=df_male_style_promt.merge(male_layered_top_df,on='type',how='left')
#
df_female_style_promt['newCategory']=df_female_style_promt.apply(lambda x: 'layered_top' if x['category'] =='top' and x['layered_top']==1.0 else x['category'],axis=1)
df_male_style_promt['newCategory']=df_male_style_promt.apply(lambda x: 'layered_top' if x['category'] =='top' and x['layered_top']==1.0 else x['category'],axis=1)

In [155]:
#
df_female_style_promt=df_female_style_promt.groupby(['styleType','newCategory'],as_index=False)['promtTypeParams'].apply(list)
df_male_style_promt=df_male_style_promt.groupby(['styleType','newCategory'],as_index=False)['promtTypeParams'].apply(list)
#
df_female_style_promt['promtTypeParams']=df_female_style_promt.apply(lambda x: {x['newCategory']:x['promtTypeParams']},axis=1)
df_male_style_promt['promtTypeParams']=df_male_style_promt.apply(lambda x: {x['newCategory']:x['promtTypeParams']},axis=1)
#
df_female_style_promt=df_female_style_promt.groupby('styleType',as_index=False)['promtTypeParams'].apply(list)
df_male_style_promt=df_male_style_promt.groupby('styleType',as_index=False)['promtTypeParams'].apply(list)
#
df_female_style_promt.to_csv('data/female_style_promt.csv')
df_male_style_promt.to_csv('data/male_style_promt.csv')



In [158]:
import pandas as pd 
from ast import literal_eval
import numpy as np
df_female_style_promt=pd.read_csv('data/female_style_promt.csv',index_col=0)
df_male_style_promt=pd.read_csv('data/male_style_promt.csv',index_col=0)
df_female_style_promt['promtTypeParams']=df_female_style_promt['promtTypeParams'].apply(literal_eval)
df_male_style_promt['promtTypeParams']=df_male_style_promt['promtTypeParams'].apply(literal_eval)


In [160]:
df_styleProportion=pd.DataFrame([{'casual':3,'classic':1,'smart-casual':2,'street':1,'old-money':1,'oversized':1,'minimalism':1}]).T.reset_index()\
    .rename(columns={'index':'styleType',0:'proportionPriority'})
df_styleProportion['proportionPriority']=df_styleProportion.proportionPriority/df_styleProportion.proportionPriority.sum()

In [161]:
df_female_style_promt=df_female_style_promt.merge(df_styleProportion,on='styleType')
df_male_style_promt=df_male_style_promt.merge(df_styleProportion,on='styleType')
df_female_style_promt['generationNumber']=(df_female_style_promt.proportionPriority*9600).astype(int)
df_male_style_promt['generationNumber']=(df_male_style_promt.proportionPriority*6400).astype(int)

In [192]:
result = {}
for d in df_female_style_promt.promtTypeParams[0]:
    result.update(d)

### https://medium.com/hackernoon/generating-the-nth-cartesian-product-e48db41bed3f    

In [200]:
import math

class LazyCartesianProduct:
    def __init__(self, sets):
        self.sets = sets
        self.divs = []
        self.mods = []
        self.maxSize = 1
        self.precompute()
    
    def precompute(self):
        for i in self.sets:
            self.maxSize = self.maxSize * len(i)
        length = len(self.sets)
        factor = 1
        for i in range((length - 1), -1, -1):
            items = len(self.sets[i])
            self.divs.insert(0, factor)
            self.mods.insert(0, items)
            factor = factor * items
    
    def entryAt(self, n):
        length = len(self.sets)
        if n < 0 or n >= self.maxSize:
            raise IndexError
        combination = []
        for i in range(0, length):
            combination.append(self.sets[i][ int(math.floor(n / self.divs[i])) % self.mods[i]])
        return combination

In [210]:
def get_clothes_combinations(promtTypeParamsValues,N):
    result = {}
    for d in promtTypeParamsValues:
        result.update(d)
    #
    first_list=[result['outerwear'],result['top'],result['bottom'],result['footwear']]
    first_cp = LazyCartesianProduct(first_list)
    first_indices=np.random.randint(0,first_cp.maxSize,size=N//2).tolist()
    first_cp_result = [first_cp.entryAt(i) for i in first_indices]
    #
    second_list=[result['layered_top'],result['top'],result['bottom'],result['footwear']]
    second_cp = LazyCartesianProduct(second_list)
    second_indices=np.random.randint(0,second_cp.maxSize,size=N//2).tolist()
    second_cp_result = [second_cp.entryAt(i) for i in second_indices]
    return first_cp_result,second_cp_result

In [364]:
get_clothes_combinations(df_male_style_promt.promtTypeParams[0],1000)[0]

[['bomber_jacket',
  'fit-oversize long_sleeve_shirt',
  'waist-regular fit-wide length-cropped jeans',
  'boots'],
 ['sleeveless_jacket',
  'fit-slim short_sleeve_shirt',
  'denim_shorts',
  'low-top_sneakers'],
 ['leather_jacket',
  'polo',
  'waist-regular fit-baggy length-cropped jeans',
  'high-top_sneakers'],
 ['denim_jacket',
  'fit-oversize linen_shirt',
  'waist-regular fit-straight length-cropped casual_trousers',
  'high-top_sneakers'],
 ['leather_jacket',
  'length-extended fit-regular neckline-V-neck t-shirt',
  'waist-regular fit-tapered length-cropped jeans',
  'boots'],
 ['bomber_jacket',
  'fit-regular long_sleeve_t-shirt',
  'waist-regular fit-wide length-cropped jeans',
  'chukka_boots'],
 ['sleeveless_jacket',
  'length-regular fit-regular neckline-V-neck t-shirt',
  'waist-high fit-wide length-cropped casual_trousers',
  'low-top_sneakers'],
 ['parka',
  'length-regular fit-regular neckline-round_neck t-shirt',
  'waist-regular fit-tapered length-cropped sweatpants

In [369]:
df_male_style_promt

,styleType,promtTypeParams,proportionPriority,generationNumber
0,casual,[{'bottom': ['fit-tapered waist-regular length...,0.3,1920
1,classic,[{'bottom': ['waist-regular fit-tapered length...,0.1,640
2,minimalism,[{'bottom': ['waist-regular fit-tapered length...,0.1,640
3,old-money,[{'bottom': ['waist-regular fit-tapered length...,0.1,640
4,oversized,[{'bottom': ['fit-tapered waist-regular length...,0.1,640
5,smart-casual,[{'bottom': ['fit-tapered waist-regular length...,0.2,1280
6,street,[{'bottom': ['fit-tapered waist-regular length...,0.1,640


In [381]:
for i in range(df_male_style_promt.shape[0]):
    combinations=get_clothes_combinations(promtTypeParamsValues=df_male_style_promt.promtTypeParams[i],
                             N=df_male_style_promt.generationNumber[i])
    #
    with open('data_combinations/cartesian_product/'+df_male_style_promt['styleType'][i]+'_outwear.json', 'w') as f:
        json.dump(combinations[0], f)
    #
    with open('data_combinations/cartesian_product/'+df_male_style_promt['styleType'][i]+'_layered_top.json', 'w') as f:
        json.dump(combinations[0], f)
    


# test

In [1]:
import pandas as pd
import json 
from ast import literal_eval
import numpy as np
from src.utils import get_df_with_clothing_promt
from src.cartesian import LazyCartesianProduct
#cartesian function
def get_clothes_combinations(promtTypeParamsValues,N):
    result = {}
    for d in promtTypeParamsValues:
        result.update(d)
    #
    first_list=[result['outerwear'],result['top'],result['bottom'],result['footwear']]
    first_cp = LazyCartesianProduct(first_list)
    first_indices=np.random.randint(0,first_cp.maxSize,size=N//2).tolist()
    first_cp_result = [first_cp.entryAt(i) for i in first_indices]
    #
    second_list=[result['layered_top'],result['top'],result['bottom'],result['footwear']]
    second_cp = LazyCartesianProduct(second_list)
    second_indices=np.random.randint(0,second_cp.maxSize,size=N//2).tolist()
    second_cp_result = [second_cp.entryAt(i) for i in second_indices]
    return first_cp_result,second_cp_result
#
with open('data/female-style-type.json') as f:
    female_style_type = json.load(f)
with open('data/male-style-type.json') as f:
    male_style_type = json.load(f)
with open('data/female-types-with-parameters.json') as f:
    female_types_with_parameters = json.load(f)
with open('data/female-types.json') as f:
    female_types = json.load(f)
with open('data/male-types-with-parameters.json') as f:
    male_types_with_parameters = json.load(f)
with open('data/male-types.json') as f:
    male_types = json.load(f)

#layered_top
female_layered_top_df=pd.read_csv('data/female_layered_top.csv',index_col=0)
male_layered_top_df=pd.read_csv('data/male_layered_top.csv',index_col=0)
# get all types by gender
female_style_type_df=pd.DataFrame(female_style_type).explode('types').reset_index(drop=True)
male_style_type_df=pd.DataFrame(male_style_type).explode('types').reset_index(drop=True)
# 
#save
female_df_clothing_promt=get_df_with_clothing_promt(gender_types=female_types,gender_types_with_parameters=female_types_with_parameters)
male_df_clothing_promt=get_df_with_clothing_promt(gender_types=male_types,gender_types_with_parameters=male_types_with_parameters)

In [2]:
male_df_clothing_promt

,category,subcategory,type,clothes_type,parameters,paramValueUnion,paramCombinations,promtTypeParams
0,top,hoodies&sweatshirts,sweatshirt,sweatshirt,"[{'name': 'fit', 'values': ['regular', 'oversi...","[[fit-regular, fit-oversize]]",[fit-regular],fit-regular sweatshirt
1,top,hoodies&sweatshirts,sweatshirt,sweatshirt,"[{'name': 'fit', 'values': ['regular', 'oversi...","[[fit-regular, fit-oversize]]",[fit-oversize],fit-oversize sweatshirt
2,top,hoodies&sweatshirts,hoodie,hoodie,"[{'name': 'fit', 'values': ['regular', 'oversi...","[[fit-regular, fit-oversize]]",[fit-regular],fit-regular hoodie
3,top,hoodies&sweatshirts,hoodie,hoodie,"[{'name': 'fit', 'values': ['regular', 'oversi...","[[fit-regular, fit-oversize]]",[fit-oversize],fit-oversize hoodie
4,top,sweaters,turtleneck,NaN,NaN,NaN,NaN,turtleneck
...,...,...,...,...,...,...,...,...
136,footwear,boots,chukka_boots,NaN,NaN,NaN,NaN,chukka_boots
137,footwear,boots,uggs_boots,NaN,NaN,NaN,NaN,uggs_boots
138,footwear,dress_shoes,oxford_shoes,NaN,NaN,NaN,NaN,oxford_shoes
139,footwear,dress_shoes,dress_shoes,NaN,NaN,NaN,NaN,dress_shoes
